### Initialization

In [77]:
# built-in
import time
from datetime import datetime
import json
import re

# third-party
import bitalino
import serial
import scientisst

In [78]:
# Initiate variables

timeout = 120
batch_size = 100
sampling_rate = 10

arduino_pattern = re.compile(r'\d+,\d\n') # pattern to make sure the message received from the serial port is in the format "{value}\n"

bitalino_args = {"mac_address": "/dev/tty.BITalino-6E-27", "sampling_rate": sampling_rate, "channels": [0, 1, 2, 3, 4, 5], "batch_size": batch_size}
scientisst_args = {"mac_address": "/dev/cu.ScientISST-54-96", "sampling_rate": sampling_rate, "channels": [1, 2, 3, 4, 5, 6], "batch_size": batch_size}
arduino_args = {"port": "/dev/cu.usbmodem12201", "sampling_rate": sampling_rate, "batch_size": batch_size}

### Define auxiliary functions

In [79]:
# Define SerialPortUnavailable exception
from serial.tools import list_ports

class SerialPortUnavailable():

    def __init__(self):
        print("Chosen port is not available. Choose one of the following ports:")
        for port in list_ports.comports():
            print(f"    * {port.name}")


SerialPortUnavailable()

Chosen port is not available. Choose one of the following ports:
    * cu.wlan-debug
    * cu.Bluetooth-Incoming-Port
    * cu.ScientISST-1D-4A
    * cu.ScientISST-54-96
    * cu.usbmodem12201


In [80]:
# Define class that hold all setup informations about the acquisition

class Acquisition:

    def __init__(self, start_time):
        
        self.sampling_rate = sampling_rate
        self.keep_alive = True
        self.devices_started = False
        self.bitalino_args = bitalino_args
        self.scientisst_args = scientisst_args
        self.arduino_args = arduino_args
        self.start_time = start_time


### Define functions for device connection

In [81]:
def connect_bitalino(mac_address):

    init_connect_time = time.time()
    print(f'Searching for BITalino... {mac_address}')

    while True:

        if (time.time() - init_connect_time) > timeout:
            print(f"    Timeout for BITalino device connection")
            raise Exception

        try:
            device = bitalino.BITalino(mac_address)
            if device.macAddress:
                break

        except Exception as e:
            print(e)
            continue

    return device

In [82]:
def connect_scientisst(mac_address):
    init_connect_time = time.time()
    print(f'Searching for ScientISST... {mac_address}')

    while True:

        if (time.time() - init_connect_time) > timeout:
            print(f"    Timeout for ScientISST device connection")
            raise Exception

        try:
            device = scientisst.ScientISST(mac_address)
            if device.address:
                break

        except Exception as e:
            print(e)
            continue

    return device

In [83]:
def connect_arduino(port):
    init_connect_time = time.time()
    print(f'Searching for Arduino... {port}')

    while True:

        if (time.time() - init_connect_time) > timeout:
            print(f"    Timeout for Arduino device connection")
            raise Exception

        try:
            device = serial.Serial(port, baudrate=115200, timeout=0)
            print("Connected!")
            break

        except Exception as e:
            print(e)
            continue

    return device    

### Define functions to start and read devices

In [84]:
def start_devices(bitalino_device, scientisst_device, acquisition):

    #bitalino_device.start(SamplingRate=acquisition.bitalino_args["sampling_rate"], analogChannels=acquisition.bitalino_args["channels"])
    scientisst_device.start(sample_rate=acquisition.scientisst_args["sampling_rate"], channels=acquisition.scientisst_args["channels"])  
    arduino_device = connect_arduino(arduino_args["port"])
    
    return arduino_device

In [85]:
def read_batch(bitalino_device, scientisst_device, arduino_device, acquisition):
    
    #bitalino_data = bitalino_device.read(acquisition.bitalino_args["batch_size"])
    scientisst_data = scientisst_device.read(matrix=True)

    arduino_data = []
    arduino_message = ""

    arduino_bytes = arduino_device.readline()
    if arduino_bytes != b"":
        arduino_data = arduino_bytes.decode()


    while len(arduino_data) < scientisst_data.shape[0]:
        arduino_bytes = arduino_device.readline() # read message from arduino and decode it from bytes to string
        
        if arduino_bytes != arduino_message:
            arduino_message = arduino_message + arduino_bytes.decode() # in case the full line comes in different messages, concatenate it
            match = arduino_pattern.search(arduino_message) # make sure the message received from the serial port is in the format "{value}\n"

        if match is not None: 
            arduino_data += [match.group().strip()]

    print(f"scientisst data:\n{scientisst_data}")
    print(f"\n\narduino data:\n{arduino_data}")


### Main script

In [86]:
try: 
    
    now = datetime.now()
    file = open(f"/Users/anasofiacc/Library/CloudStorage/OneDrive-UniversidadedeLisboa/PhD/Clynx project/Respiration sensor/magnetometer/code/sample_{now.strftime('%d-%m-%Y_%H-%M-%S')}.txt","w")
    acquisition = Acquisition(time.time(), file)

    # Connect devices
    scientisst_device = connect_scientisst(scientisst_args["mac_address"])
    #bitalino_device = connect_bitalino(bitalino_args["mac_address"])
    bitalino_device = None
    arduino_device = None

except KeyboardInterrupt:
    print(f"Script terminated before acquisition was started")
    acquisition.keep_alive = False

try: 

    while acquisition.keep_alive == True:
        
        if not acquisition.devices_started: 
            # If the devices have not yet started acquiring or they are paused, start acquisition
            try:
                arduino_device = start_devices(bitalino_device, scientisst_device, acquisition)
                acquisition.devices_started = True

            except Exception as e:
                print(e)
                pass
        
        else:
            read_batch(bitalino_device, scientisst_device, arduino_device, acquisition)

except KeyboardInterrupt:
    #bitalino_device.stop()
    scientisst_device.stop()
    arduino_device.close()
    print(f"Acquisition terminated")


Searching for ScientISST... /dev/cu.ScientISST-54-96
Connecting to /dev/cu.ScientISST-54-96...
ScientISST version: 1.0
ScientISST Board Vref: 1114
ScientISST Board ADC Attenuation Mode: 0
Connected!
Searching for Arduino... /dev/cu.usbmodem12201
Connected!
2
scientisst data:
[[  0   0   0   0   0   0 254   0 254   0 254   0 254   0 254   0 254]
 [  1   0   0   0   0   0 254   0 254   0 254   0 254   0 254   0 254]]


arduino data:
[]
2
scientisst data:
[[  2   0   0   0   0   0 254   0 254   0 254   0 254   0 254   0 254]
 [  3   0   0   0   0   0 254   0 254   0 254   0 254   0 254   0 254]]


arduino data:
[]
2
scientisst data:
[[  4   0   0   0   0   0 254   0 254   0 254   0 254   0 254   0 254]
 [  5   0   0   0   0   0 254   0 254   0 254   0 254   0 254   0 254]]


arduino data:
[]
2
scientisst data:
[[  6   0   0   0   0   0 254   0 254   0 254   0 254   0 254   0 254]
 [  7   0   0   0   0   0 254   0 254   0 254   0 254   0 254   0 254]]


arduino data:
[]
2
scientisst data:
